In [1]:
#from pyiceberg.catalog.hive import HiveCatalog
#hive_catalog = HiveCatalog(
#    "hive",
#    **{
#        "uri": "thrift://hive-metastore:9083",
#        "s3.endpoint": "http://ozone-s3g:9878",
#        "py-io-impl": "pyiceberg.io.fsspec.FsspecFileIO", #"pyiceberg.io.pyarrow.PyArrowFileIO", #
#        "s3.access-key-id": "ozone-access-key",
#        "s3.secret-access-key": "ozone-secret-key",
#        "warehouse": "s3a://mybucket/warehouse/",
#        "s3.path-style-access": "true",
#    }
#)

In [2]:
from pyiceberg.catalog import load_catalog

from pyiceberg.schema import Schema
from pyiceberg.types import (
    IntegerType,
    StringType,
    NestedField,
)
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import IdentityTransform

In [3]:
catalog = load_catalog(
    "rest",
    **{
        "uri": "http://nessie-catalog:19120/iceberg",
        "s3.endpoint": "http://ozone-s3g:9878",
        "py-io-impl": "pyiceberg.io.fsspec.FsspecFileIO", #"pyiceberg.io.pyarrow.PyArrowFileIO", #
        "s3.access-key-id": "ozone-access-key",
        "s3.secret-access-key": "ozone-secret-key",
        "warehouse": "warehouse",
        "s3.path-style-access": "true",
        "authentication.type": "NONE",
    }
)

In [5]:
catalog.drop_namespace("test_iceberg_db")

In [6]:
catalog.create_namespace(
    "test_iceberg_db",
    properties={
        "location": "s3a://mybucket/iceberg/test_iceberg_db/"
    }
)

In [7]:
table_schema = Schema(
    NestedField(field_id=1, name="id", field_type=IntegerType(), required=False),
    NestedField(field_id=2, name="descr", field_type=StringType(), required=False),
    NestedField(field_id=3, name="report_dt", field_type=StringType(), required=False),
)

table_partition = PartitionSpec(
    PartitionField(source_id=3, field_id=1000, transform=IdentityTransform(), name="report_dt")
)

In [4]:
catalog.drop_table(identifier=("test_iceberg_db", "test_iceberg_table"))

In [8]:
catalog.create_table(
    identifier=("test_iceberg_db", "test_iceberg_table"),
    schema=table_schema,
    partition_spec=table_partition,
    properties={
        "owner": "MegaSuper_DE",
        "write.format.default": "parquet",
        "write.parquet.compression-codec": "snappy",
        #"external": "true",
        #"table_type": "EXTERNAL_TABLE",
    }
)

test_iceberg_table(
  1: id: optional int,
  2: descr: optional string,
  3: report_dt: optional string
),
partition by: [report_dt],
sort order: [],
snapshot: null

In [9]:
table = catalog.load_table(("test_iceberg_db", "test_iceberg_table"))

In [10]:
table

test_iceberg_table(
  1: id: optional int,
  2: descr: optional string,
  3: report_dt: optional string
),
partition by: [report_dt],
sort order: [],
snapshot: null